# Import Packages

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

# Load Data

In [2]:
df = pd.read_csv("NASDAQ_test.csv")
SPY = pd.read_csv("SPY.csv")
Rf = pd.read_csv("Risk_Free.csv")

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
df.head(3)

,PERMNO,Date,TICKER,Price,Return,Shares Outstanding,Turnover,Dollar Turnover,Current Ratio,Cash Ratio,Quick Ratio,Revenue Growth,COGS Growth,Operating Income Growth,Net Income Growth,Net Operating Cash Flow Growth
0,10002,2011-01-01,BTFG,2.78,0.041199,17.515,41747.87325,111466.8216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10002,2011-02-01,BTFG,2.64,-0.050360,17.515,38784.18498,107820.0343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10002,2011-03-01,BTFG,2.46,-0.068182,17.785,33212.67485,87681.4616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
len(df)

389543

In [5]:
df1 = df.copy()[['PERMNO', 'Date', 'TICKER', 'Return']]  # Save time

In [6]:
df1.head(3)

,PERMNO,Date,TICKER,Return
0,10002,2011-01-01,BTFG,0.041199
1,10002,2011-02-01,BTFG,-0.050360
2,10002,2011-03-01,BTFG,-0.068182


In [7]:
SPY.head(3)

,Date,SPY Return
0,1990-02-01,0.0085
1,1990-03-01,0.0243
2,1990-04-01,-0.0269


In [8]:
Rf.head(3)

,Date,Rf
0,1990-01-01,0.006958
1,1990-02-01,0.007033
2,1990-03-01,0.007208


# Rolling Window and Calculate Var & Cov

In [9]:
df1 = pd.merge(df1, SPY, on = 'Date', how = 'outer')
df1 = pd.merge(df1, Rf, on = 'Date', how = 'outer')

In [10]:
df1.dropna(subset=['TICKER'], inplace=True)

In [11]:
len(df1)

389443

In [12]:
df1.head(3)

,PERMNO,Date,TICKER,Return,SPY Return,Rf
0,10002.0,2011-01-01,BTFG,0.041199,0.0226,0.001627
1,10025.0,2011-01-01,AEPI,0.067052,0.0226,0.001627
2,10026.0,2011-01-01,JJSF,-0.119610,0.0226,0.001627


In [13]:
df1.reset_index(inplace=True, drop=True)

In [14]:
def rolling(df_):
    df = df_.copy()
    TICKER = list(set(df['TICKER']))
    dataframe = df[df['TICKER'] == TICKER[0]].copy()
    dataframe['Var'] = dataframe['SPY Return'].rolling(window=36).var()
    dataframe['Cov'] = dataframe['Return'].rolling(window=36).cov(dataframe['SPY Return'])
    dataframe['Beta'] = dataframe['Cov'] / dataframe['Var']
    dataframe['Alpha'] = dataframe['Return'] - dataframe['Rf'] - dataframe['Beta'] * (dataframe['SPY Return'] - dataframe['Rf'])
    
    for i in range(1, len(TICKER)):
        #DF = df.loc[TICKER[i]]
        DF = df[df['TICKER'] == TICKER[i]].copy()
        if len(DF) < 36:
            continue
        DF['Var'] = DF['SPY Return'].rolling(window=36).var()
        DF['Cov'] = DF['Return'].rolling(window=36).cov(DF['SPY Return'])
        DF['Beta'] = DF['Cov'] / DF['Var']
        DF['Alpha'] = DF['Return'] - DF['Rf'] - DF['Beta'] * (DF['SPY Return'] - DF['Rf']) 
        
        dataframe = pd.concat([dataframe, DF])
        
    return dataframe

In [15]:
df2 = rolling(df1)

In [16]:
df2.reset_index(inplace=True, drop=True)
df2

,PERMNO,Date,TICKER,Return,SPY Return,Rf,Var,Cov,Beta,Alpha
0,18896.0,2019-08-01,INMD,NaN,-0.0181,0.001159,NaN,NaN,NaN,NaN
1,18896.0,2019-08-01,INMD,NaN,-0.0181,0.001159,NaN,NaN,NaN,NaN
2,18896.0,2019-09-01,INMD,0.080905,0.0172,0.001293,NaN,NaN,NaN,NaN
3,18896.0,2019-09-01,INMD,0.080905,0.0172,0.001293,NaN,NaN,NaN,NaN
4,18896.0,2019-10-01,INMD,0.430033,0.0204,0.001269,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
374154,10297.0,2016-11-01,LSBG,0.196487,0.0342,0.001528,0.001055,0.000493,0.467546,0.179683
374155,10297.0,2016-12-01,LSBG,0.082110,0.0182,0.001574,0.001058,0.000510,0.482036,0.072522
374156,10297.0,2016-12-01,LSBG,0.082110,0.0182,0.001574,0.001026,0.000477,0.464998,0.072805
374157,10297.0,2017-01-01,LSBG,-0.069945,0.0179,0.001590,0.000960,0.000468,0.487776,-0.079491


In [17]:
df3 = df2.copy()[['PERMNO', 'Date', 'Beta', 'Alpha']]

In [18]:
df3.head(3)

,PERMNO,Date,Beta,Alpha
0,18896.0,2019-08-01,NaN,NaN
1,18896.0,2019-08-01,NaN,NaN
2,18896.0,2019-09-01,NaN,NaN


In [26]:
df4 = pd.merge(df, df3, on = ['PERMNO', 'Date'], how = 'outer')

In [27]:
df4

,PERMNO,Date,TICKER,Price,Return,Shares Outstanding,Turnover,Dollar Turnover,Current Ratio,Cash Ratio,Quick Ratio,Revenue Growth,COGS Growth,Operating Income Growth,Net Income Growth,Net Operating Cash Flow Growth,Beta,Alpha
0,10002,2011-01-01,BTFG,2.78,0.041199,17.515,4.174787e+04,1.114668e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10002,2011-02-01,BTFG,2.64,-0.050360,17.515,3.878418e+04,1.078200e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10002,2011-03-01,BTFG,2.46,-0.068182,17.785,3.321267e+04,8.768146e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10002,2011-04-01,BTFG,2.50,0.016260,17.785,3.694248e+04,9.087850e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10002,2011-05-01,BTFG,2.44,-0.024000,17.785,1.160360e+04,2.900900e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633118,93436,2019-11-01,TSLA,329.94,0.047695,180.000,1.289401e+06,4.060581e+08,1.078257,0.549083,0.725310,0.398156,0.382514,-50.008532,-0.418439,-1.350216,0.194220,0.040003
633119,93436,2019-12-01,TSLA,418.33,0.267897,181.000,8.752726e+05,2.887874e+08,1.134621,0.610668,0.801631,-0.007373,-0.055517,0.499310,-1.351352,3.375000,0.273721,0.259044
633120,93436,2019-12-01,TSLA,418.33,0.267897,181.000,8.752726e+05,2.887874e+08,1.134621,0.610668,0.801631,-0.007373,-0.055517,0.499310,-1.351352,3.375000,0.351416,0.256931
633121,93436,2019-12-01,TSLA,418.33,0.267897,181.000,8.752726e+05,2.887874e+08,1.134621,0.610668,0.801631,-0.007373,-0.055517,0.499310,-1.351352,3.375000,0.273721,0.259044


# Drop Missing Values

In [37]:
df5 = df4.dropna(how='any')

In [38]:
len(df5)

349697

In [39]:
df5 

,PERMNO,Date,TICKER,Price,Return,Shares Outstanding,Turnover,Dollar Turnover,Current Ratio,Cash Ratio,Quick Ratio,Revenue Growth,COGS Growth,Operating Income Growth,Net Income Growth,Net Operating Cash Flow Growth,Beta,Alpha
61,10025,2013-12-01,AEPI,52.83,0.024631,5.601,3.744196e+05,1.930507e+07,2.022299,0.114762,1.185726,0.022057,0.025081,0.049671,0.686792,13.872143,1.435235,-0.008607
62,10025,2014-01-01,AEPI,44.04,-0.166383,5.602,4.491568e+05,2.372895e+07,1.864672,0.011480,0.996758,0.025319,0.030535,0.053156,-0.446309,-0.111736,1.552872,-0.110405
63,10025,2014-02-01,AEPI,42.62,-0.032243,5.602,2.942260e+05,1.295771e+07,1.864672,0.011480,0.996758,0.025319,0.030535,0.053156,-0.446309,-0.111736,1.497296,-0.096150
64,10025,2014-03-01,AEPI,37.10,-0.129517,5.602,1.773724e+05,7.559610e+06,1.864672,0.011480,0.996758,0.025319,0.030535,0.053156,-0.446309,-0.111736,1.522024,-0.139266
65,10025,2014-04-01,AEPI,35.62,-0.039892,5.081,1.421912e+05,5.275293e+06,2.228061,0.038266,1.253722,-0.089373,-0.055983,-0.587297,-4.764646,-0.891072,1.533606,-0.048653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633118,93436,2019-11-01,TSLA,329.94,0.047695,180.000,1.289401e+06,4.060581e+08,1.078257,0.549083,0.725310,0.398156,0.382514,-50.008532,-0.418439,-1.350216,0.194220,0.040003
633119,93436,2019-12-01,TSLA,418.33,0.267897,181.000,8.752726e+05,2.887874e+08,1.134621,0.610668,0.801631,-0.007373,-0.055517,0.499310,-1.351352,3.375000,0.273721,0.259044
633120,93436,2019-12-01,TSLA,418.33,0.267897,181.000,8.752726e+05,2.887874e+08,1.134621,0.610668,0.801631,-0.007373,-0.055517,0.499310,-1.351352,3.375000,0.351416,0.256931
633121,93436,2019-12-01,TSLA,418.33,0.267897,181.000,8.752726e+05,2.887874e+08,1.134621,0.610668,0.801631,-0.007373,-0.055517,0.499310,-1.351352,3.375000,0.273721,0.259044


In [40]:
df5 = pd.merge(df5, SPY, on = 'Date', how = 'outer')
df5 = pd.merge(df5, Rf, on = 'Date', how = 'outer')

df5.dropna(how='any', inplace=True)

In [45]:
df6 = df5.copy()
df6['Date'] = pd.to_datetime(df6['Date'])
df6 = df6.set_index('Date')

In [51]:
df7 = df6['2015':'2019'].copy()

In [52]:
df7.reset_index(inplace=True)
df7.to_csv("df_test.csv", index=False)